In [1]:
import torch, time, pickle
import torch_geometric as tg
from torch_geometric.data import Data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import os.path as osp
import networkx as nx
path='../../../../tigress/mcranmer/merger_trees/isotrees/'
def convert(d,p):
    dfin=[]
    if len(p)!=len(np.unique(p)):
        print('Wrong order of prog/desc')
    else:
        no=d[0]
        for desc in d:
            if desc==no:
                dfin.append(0)
            else:
                dfin.append(p.index(desc)+1)
    return dfin, np.arange(1, 1+len(p))

In [2]:
# cols=np.array([0,2,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,35]+list(range(37,60)))
is_cat=[0,1,0,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,\
        0,0,0,0,0,0,0,0,0,0,0]
cols=np.array([0,2,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,35]+list(range(37,60)))
# cols=np.array([0,2,4,5,6,7,8,10])
case='222_all'
is_cat=np.array([bool(i) for i in is_cat])
i0,i1=0,2
j0,j1=0,2
k0,k1=0,2
scaler = pickle.load(open('transformer0.pkl', 'rb'))

In [ ]:
dat=[]
for i in range(i0,i1+1):
    for j in range(j0,j1+1):
        for k in range(k0,k1+1):
            print(f'Loading isotree {i}_{j}_{k}')
            start=time.time()
            pd1=pd.read_table(path+f'isotree_{i}_{j}_{k}.dat', skiprows=0, delimiter='\s+') # consider enforcing dtypes upon import
            print(f'isotree {i}_{j}_{k} loaded, restructuring')
            raw=pd1.drop(axis=0, index=np.arange(50)).reset_index()
            trees=raw[raw.isna()['A[z]']] 
            halos=raw[~raw.isna()['A[z]']] 
            del raw
            del pd1
            f,it="float64", "int64"
            cs=[f,it,f,it,it,it,it,it,it,f,f,f,f,f,it,f,f,f,f,f,f,f,f,f,f,f]
            dicts = {}
            keys = halos.columns[1:26]
            castto = cs
            for d, key in enumerate(keys):
                    dicts[key] = castto[d]
            halos=halos.astype(dicts)
            
            hard=[0,2,15]
            def logit(x):
                return np.log10((x+0.001)/(1.01-x))
            ##logit transform the hards
            for h in hard:
                halos[halos.columns[h+1]]=logit(halos[halos.columns[h+1]])
            log=[10,38, 39, 40, 41, 42]

            def logt(x):
                return np.log10(x+1)
            for l in log:
                halos[halos.columns[l+1]]=logt(halos[halos.columns[l+1]])
            tcols=np.array(cols[~is_cat[cols]])+1
#             halos[halos.columns[tcols]]=scaler.transform(halos[halos.columns[tcols]])
            
            print(f'Splitting to tree')
            spli=np.split(np.array(halos)[:,1:], np.array(trees.iloc[1:].index)-np.arange(1,len(trees.index)))
            splits=[]
            for s in spli:
                if s[0,10]>10:
                    splits.append(s)
            splits=np.array(splits, dtype=object)
            split=[]
            for tree in splits:
                s=tree[np.logical_or(tree[:,3] == -1,tree[:,4]!=1)]
                split.append(s)
            split=np.array(split, dtype=object)
            print('Split done')
            print('Loading targets')
            samp='~/../../../tigress/mcranmer/merger_trees/samout/'
            ex=f'{i}_{j}_{k}/'
            ex+='galprop_0-99.dat'
            pdc=pd.read_table(samp+ex, skiprows=0, delimiter=',', nrows=41, header=None)
            newcols=pdc.iloc[:,0]
            pds=pd.read_table(samp+ex, skiprows=41, delimiter='\s+', header=None)
            pds.columns=np.array(newcols)
            pd0=pds[pds[pds.columns[3]]==0.00] # subhaloes
            pdcen=pd0[(pd0[pds.columns[1]]==pd0[pds.columns[2]])] ##central haloes
            
            rhalid=np.array(pdcen[pds.columns[2]])
            del pd0
            del pds
            halwgal=[]
            ids=[]
            out=[]
            for idx, tree in enumerate(split):
                if tree[0,1] in rhalid:
                    halwgal.append(tree)
                    ids.append(idx)
                    index=np.where(rhalid==tree[0,1])
                    out.append(np.array(pdcen.iloc[index])[0][8]) ## target variable
            del pdcen
            stop=time.time()
            hraw=np.array(splits[ids], dtype=object)
            hals=[]
            pr,de=[],[]
            print('Making merger tree')
            for n in tqdm(range(len(halwgal))):
                h=halwgal[n]
                roots=h[h[:,4]==0]
            #     mergers=h[np.logical_and(h[:,4]>1,h[:,3]!=-1)] #remove self-loop, is broken since some roots go directly to final
                mergers=h[h[:,4]>1]
                final=h[h[:,3]==-1]
                pro, des=[],[]
                for r in roots:
                    descid=hraw[n][:,3][np.where(r[1]==hraw[n][:,1])]
                    while descid not in mergers[:,1]:
                        descid=hraw[n][:,3][np.where(descid==hraw[n][:,1])] #consider adding the number of steps it went through
                    pro.append(r[1])
                    des.append(descid[0])
                m=mergers[mergers[:, 0].argsort()] #sort by redshift, starting at early times
                mdes,mpro=list(m[:,1][1:]), list(m[:,1])
                des+=mdes
                pro+=mpro
                des.append(final[:,1][0])
                des,pro=convert(des[::-1],pro[::-1])
                hal2=np.vstack([final,np.flip(m,axis=0).copy(),np.flip(roots,axis=0).copy()])
                hal2=hal2[:,cols] 
                hals.append(hal2)
                pr.append([int(p) for p in pro])
                de.append([int(d) for d in des])
            hals=np.array(hals,dtype=object)
            out=np.log(out)/5 #homemade scaling
            for n in tqdm(range(len(out))):
                edge_index = torch.tensor([pr[n],de[n]], dtype=torch.long)
                x = torch.tensor(hals[n], dtype=torch.float)

                y=torch.tensor(out[n], dtype=torch.float)
                graph=Data(x=x, edge_index=edge_index, y=y)
                dat.append(graph)
            stop=time.time()
            print(f'Done with tree. \n Time elapsed {stop-start} s')
print("Saving dataset")
if not osp.exists(f'../../../../scratch/gpfs/cj1223/GraphStorage/{case}'):
    os.mkdir(f'../../../../scratch/gpfs/cj1223/GraphStorage/{case}')
                  
with open(f'../../../../scratch/gpfs/cj1223/GraphStorage/{case}/data.pkl', 'wb') as handle:
    pickle.dump(dat, handle)

Loading isotree 0_0_0


/home/cj1223/.conda/envs/juptorch/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


isotree 0_0_0 loaded, restructuring
Splitting to tree
Split done
Loading targets
Making merger tree


100%|███████████████████████████████████████████████████████████| 769/769 [00:00<00:00, 2763.68it/s]


Done with tree. 
 Time elapsed 172.76603198051453 s
Loading isotree 0_0_1
isotree 0_0_1 loaded, restructuring
Splitting to tree
Split done
Loading targets
Making merger tree


...

100%|█████████████████████████████████████████████████████████| 1476/1476 [00:00<00:00, 5600.38it/s]


Done with tree. 
 Time elapsed 1209.3880019187927 s
Loading isotree 0_1_0
isotree 0_1_0 loaded, restructuring
Splitting to tree
Split done
Loading targets
Making merger tree


100%|█████████████████████████████████████████████████████████| 1128/1128 [00:00<00:00, 8979.84it/s]


Done with tree. 
 Time elapsed 664.5416121482849 s
Loading isotree 0_1_1
isotree 0_1_1 loaded, restructuring
Splitting to tree
Split done
Loading targets
Making merger tree


 79%|██████████████████████████████████████████████▍            | 1526/1937 [13:54<00:52,  7.84it/s]

In [ ]:
print(f'Loading isotree {i}_{j}_{k}')
start=time.time()
pd1=pd.read_table(path+f'isotree_{i}_{j}_{k}.dat', skiprows=0, delimiter='\s+') # consider enforcing dtypes upon import
print(f'isotree {i}_{j}_{k} loaded, restructuring')
raw=pd1.drop(axis=0, index=np.arange(50)).reset_index()
trees=raw[raw.isna()['A[z]']] 
halos=raw[~raw.isna()['A[z]']] 
f,it="float64", "int64"
cs=[f,it,f,it,it,it,it,it,it,f,f,f,f,f,it,f,f,f,f,f,f,f,f,f,f,f]
dicts = {}
keys = halos.columns[1:26]
castto = cs
for d, key in enumerate(keys):
        dicts[key] = castto[d]
halos=halos.astype(dicts)

hard=[0,2,15]
def logit(x):
    return np.log10((x+0.001)/(1.01-x))
##logit transform the hards
for h in hard:
    halos[halos.columns[h+1]]=logit(halos[halos.columns[h+1]])
log=[10,38, 39, 40, 41, 42]

def logt(x):
    return np.log10(x+1)
for l in log:
    halos[halos.columns[l+1]]=logt(halos[halos.columns[l+1]])
tcols=np.array(cols[~is_cat[cols]])+1
halos[halos.columns[tcols]]=scaler.transform(halos[halos.columns[tcols]])



In [ ]:
print(f'Restructured, splitting to tree')
spli=np.split(np.array(halos)[:,1:], np.array(trees.iloc[1:].index)-np.arange(1,len(trees.index)))
splits=[]
for s in spli:
    if s[0,10]>1:
        splits.append(s)
splits=np.array(splits, dtype=object)
split=[]
for tree in splits:
    s=tree[np.logical_or(tree[:,3] == -1,tree[:,4]!=1)]
    split.append(s)
split=np.array(split, dtype=object)
print('Split done')
print(len(split))
print('Loading targets')

In [ ]:
samp='~/../../../tigress/mcranmer/merger_trees/samout/'
ex=f'{0}_{j}_{k}/'
ex+='galprop_0-99.dat'
pdc=pd.read_table(samp+ex, skiprows=0, delimiter=',', nrows=41, header=None)
newcols=pdc.iloc[:,0]
pds=pd.read_table(samp+ex, skiprows=41, delimiter='\s+', header=None)
pds.columns=np.array(newcols)
pd0=pds[pds[pds.columns[3]]==0.00] # subhaloes
pdcen=pd0[(pd0[pds.columns[1]]==pd0[pds.columns[2]])] ##central haloes

rhalid=np.array(pdcen[pds.columns[2]])
halwgal=[]
ids=[]
out=[]
for idx, tree in enumerate(split):
    if tree[0,1] in rhalid:
        halwgal.append(tree)
        ids.append(idx)
        index=np.where(rhalid==tree[0,1])
        out.append(np.array(pdcen.iloc[index])[0][8]) ## target variable
stop=time.time()
hraw=np.array(splits[ids], dtype=object)
len(hraw)

In [ ]:
hals=[]
pr,de=[],[]
print('Making merger tree')
for i in tqdm(range(len(halwgal))):
    h=halwgal[i]
    roots=h[h[:,4]==0]
#     mergers=h[np.logical_and(h[:,4]>1,h[:,3]!=-1)] #remove self-loop, is broken since some roots go directly to final
    mergers=h[h[:,4]>1]
    final=h[h[:,3]==-1]
    pro, des=[],[]
    for r in roots:
        descid=hraw[i][:,3][np.where(r[1]==hraw[i][:,1])]
        while descid not in mergers[:,1]:
            descid=hraw[i][:,3][np.where(descid==hraw[i][:,1])] #consider adding the number of steps it went through
        pro.append(r[1])
        des.append(descid[0])
    m=mergers[mergers[:, 0].argsort()] #sort by redshift, starting at early times
    mdes,mpro=list(m[:,1][1:]), list(m[:,1])
    des+=mdes
    pro+=mpro
    des.append(final[:,1][0])
    des,pro=convert(des[::-1],pro[::-1])
    hal2=np.vstack([final,np.flip(m,axis=0).copy(),np.flip(roots,axis=0).copy()])
    hal2=hal2[:,cols] 
    hals.append(hal2)
    pr.append([int(p) for p in pro])
    de.append([int(d) for d in des])
hals=np.array(hals,dtype=object)
for n in tqdm(range(len(out))):
    edge_index = torch.tensor([pr[n],de[n]], dtype=torch.long)
    x = torch.tensor(hals[n], dtype=torch.float)

    y=torch.tensor(out[n], dtype=torch.float)
    graph=Data(x=x, edge_index=edge_index, y=y)
    dat.append(graph)
stop=time.time()
print(f'Done with tree. \n Time elapsed {stop-start} s')